In [31]:
import os
os.chdir(r"E:\data\courses\krish naik\MLOPS\DS prj")
%pwd

'E:\\data\\courses\\krish naik\\MLOPS\\DS prj'

In [32]:
os.environ['MLFLOW_TRACKING_URI'] = "https://dagshub.com/ayushpoddar351/Wine-Quality-Prediction.mlflow"
os.environ['MLFLOW_TRACKING_USERNAME'] = "ayushpoddar351"
os.environ['MLFLOW_TRACKING_PASSWORD'] = "8a2baa812e0371675e184ebf7debdd38ec10d1ee"

In [33]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [34]:
from src.DataScienceProject.constants import *
from src.DataScienceProject.utils.common import read_yaml, create_directories, save_json

class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH, schema_filepath = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri = "https://dagshub.com/ayushpoddar351/Wine-Quality-Prediction.mlflow"
        )
        return model_evaluation_config



In [35]:
import os, pandas as pd
import numpy as np, joblib
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn



class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self,actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2

    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            pred_y = model.predict(test_x)
            (rmse, mae, r2) = self.eval_metrics(test_y, pred_y)
            # Saving metrics as local
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("mae", mae)
            mlflow.log_metric("r2", r2)

            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(sk_model=model, name="ElasticnetModel")
            else:
                mlflow.sklearn.log_model(model, "model")




In [36]:
try:
    config_manager = ConfigurationManager()
    model_evaluation_config = config_manager.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2025-08-01 18:47:34,926: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-08-01 18:47:34,928: INFO: common: yaml file: params.yaml loaded successfully]
[2025-08-01 18:47:34,930: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-08-01 18:47:34,931: INFO: common: created directory at: artifacts]
[2025-08-01 18:47:34,932: INFO: common: created directory at: artifacts/model_evaluation]
[2025-08-01 18:47:35,356: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]
🏃 View run intrigued-gnat-205 at: https://dagshub.com/ayushpoddar351/Wine-Quality-Prediction.mlflow/#/experiments/0/runs/d05d08e719e2488099bbd4bb7d486c17
🧪 View experiment at: https://dagshub.com/ayushpoddar351/Wine-Quality-Prediction.mlflow/#/experiments/0


RestException: INTERNAL_ERROR: Response: {'error': 'unsupported endpoint, please contact support@dagshub.com'}